In [ ]:
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings

import pandas as pd
import numpy as np
from typing import List, Union
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss

import os 
import sys
sys.path.append(os.path.abspath("../src"))
from core.ComplaintVectorPipeline import ComplaintVectorPipeline

In [ ]:
initial_filtered_data = pd.read_csv("../data/processed/filtered_complaints.csv")
print("Data loaded Successfully ✅✅✅")

Data loaded Successfully ✅✅✅


In [25]:
# Instantiate our pipeline
pipeline = ComplaintVectorPipeline()
print("pipeline instantiated successfully✅✅✅")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pipeline instantiated successfully✅✅✅


In [26]:
# Run the chunking process
chunk_df = pipeline.preprocess_and_chunk(
    initial_filtered_data,
    text_column="Normalized Consumer complaint narrative",
    min_words=20,
    metadata_fields=[]  # Leave this empty for now unless we want extra fields later
)
print("\n\n\n\n Preprocessing and Chunking Done✅✅✅")





 Preprocessing and Chunking Done✅✅✅


In [28]:
# Display the chunked dataframe and its shape
print(chunk_df.shape)
chunk_df

(628990, 8)


,source_row,chunk_id,chunk_text,word_count,Complaint ID,Product,Issue,Sub-issue
0,0,0,a card was opened under my name by a fraudster...,82,14069121,Credit card,Getting a credit card,Card opened without my consent or knowledge
1,1,0,i made the mistake of using my wellsfargo debi...,105,14061897,Savings account,Managing an account,Deposits and withdrawals
2,2,0,"dear cfpb, i have a secured credit card with c...",156,14047085,Credit card,"Other features, terms, or problems",Other problem
3,3,0,i have a citi rewards cards. the credit balanc...,136,14040217,Credit card,Incorrect information on your report,Account information incorrect
4,3,1,. i contacted them and all they said was consu...,107,14040217,Credit card,Incorrect information on your report,Account information incorrect
...,...,...,...,...,...,...,...,...
628985,327498,0,"i already have a case , but it should not have...",138,6238646,Savings account,Managing an account,Deposits and withdrawals
628986,327498,1,". instead, the account was left open for multi...",143,6238646,Savings account,Managing an account,Deposits and withdrawals
628987,327498,2,. wells fargo was negligent in that regard sin...,132,6238646,Savings account,Managing an account,Deposits and withdrawals
628988,327498,3,. the consumer fraud comment from the associat...,141,6238646,Savings account,Managing an account,Deposits and withdrawals


In [29]:
# The new cleaned filtered complaints dataframe which includes only complaints greater than 20 word count
pipeline.filtered_df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,word_count,Target_Product,Normalized Consumer complaint narrative,chunks
0,2025-06-13,Credit card,Store credit card,Getting a credit card,Card opened without my consent or knowledge,A XXXX XXXX card was opened under my name by a...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,78230,...,Web,2025-06-13,Closed with non-monetary relief,Yes,NaN,14069121,91,Credit card,a card was opened under my name by a fraudster...,[a card was opened under my name by a fraudste...
1,2025-06-13,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,I made the mistake of using my wellsfargo debi...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,ID,83815,...,Web,2025-06-13,Closed with explanation,Yes,NaN,14061897,109,Savings account,i made the mistake of using my wellsfargo debi...,[i made the mistake of using my wellsfargo deb...
2,2025-06-12,Credit card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,"Dear CFPB, I have a secured credit card with c...",Company has responded to the consumer and the ...,"CITIBANK, N.A.",NY,11220,...,Web,2025-06-13,Closed with monetary relief,Yes,NaN,14047085,156,Credit card,"dear cfpb, i have a secured credit card with c...","[dear cfpb, i have a secured credit card with ..."
3,2025-06-12,Credit card,General-purpose credit card or charge card,Incorrect information on your report,Account information incorrect,I have a Citi rewards cards. The credit balanc...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",IL,60067,...,Web,2025-06-12,Closed with explanation,Yes,NaN,14040217,233,Credit card,i have a citi rewards cards. the credit balanc...,[i have a citi rewards cards. the credit balan...
4,2025-06-09,Credit card,General-purpose credit card or charge card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,b'I am writing to dispute the following charge...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,78413,...,Web,2025-06-09,Closed with monetary relief,Yes,NaN,13968411,454,Credit card,b'i am writing to dispute the following charge...,[b'i am writing to dispute the following charg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327494,2022-11-24,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,US bank is holding a {$200.00} Check Deposit w...,Company has responded to the consumer and the ...,U.S. BANCORP,IN,47331,...,Web,2022-11-24,Closed with explanation,Yes,NaN,6237410,94,Savings account,us bank is holding a $ check deposit with no e...,[us bank is holding a $ check deposit with no ...
327495,2022-08-19,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,"I had over 80 small charges, many were for the...",NaN,PNC Bank N.A.,KY,40205,...,Web,2022-08-19,Closed with explanation,Yes,NaN,5895804,118,Savings account,"i had over 80 small charges, many were for the...","[i had over 80 small charges, many were for th..."
327496,2022-06-06,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Confusing or missing disclosures,NaN,"I moved into XXXX XXXX XXXX XXXX, owned, and ...",Company believes complaint represents an oppor...,Entrata Inc.,CA,922XX,...,Web,2022-06-06,Closed with monetary relief,Yes,NaN,5638610,387,Money transfers,"i moved into , owned, and operated by on . eac...","[i moved into , owned, and operated by on . ea..."
327497,2022-08-19,"Money transfer, virtual currency, or money ser...",International money transfer,Other transaction problem,NaN,"Hello, I'm an XXXX of XXXX XXXX company that ...",Company has respond

In [32]:
# Lets embed our chunked text
embeddings = pipeline.embed_chunks()
print("Embedding Done✅✅✅")

Batches:   0%|          | 0/9828 [00:00<?, ?it/s]

Embedding Done✅✅✅


In [33]:
print(pipeline.chunk_df["embedding"].iloc[0][:5])  # preview first 5 dims of first embedding
print(f"Embedding shape: {len(pipeline.chunk_df['embedding'][0])}")

[-0.042777352035045624, 0.025624385103583336, -0.07883816957473755, 0.022501634433865547, -0.009484909474849701]
Embedding shape: 384


In [35]:
#Display the embedding of the chunk_df
pipeline.chunk_df["embedding"]

0         [-0.042777352035045624, 0.025624385103583336, ...
1         [-0.04982152208685875, 0.08977359533309937, 0....
2         [-0.035176705569028854, 0.011623036116361618, ...
3         [-0.08906636387109756, 0.03862784057855606, -0...
4         [-0.03665914759039879, -0.02310161292552948, 0...
                                ...                        
628985    [-0.04320886358618736, -0.021847466006875038, ...
628986    [-0.052816007286310196, -0.012807104736566544,...
628987    [-0.06271518766880035, 0.01023461390286684, 0....
628988    [-0.10470777750015259, -0.004357500467449427, ...
628989    [-0.053923919796943665, 0.13369841873645782, 0...
Name: embedding, Length: 628990, dtype: object

In [36]:
# Build the FAISS index using the embedded vectors
faiss_index = pipeline.build_faiss_index()

In [37]:
# Check if indexing was successfully done 
print("🔍 FAISS index built:", isinstance(pipeline.index, faiss.IndexFlatIP))
print("   → Total vectors in index:", pipeline.index.ntotal)
print("   → Expected vector shape:", pipeline.index.d)

🔍 FAISS index built: True
   → Total vectors in index: 628990
   → Expected vector shape: 384


In [38]:
# Check for Embedding
print("📊 Sample embedding (first 5 dims):", pipeline.chunk_df["embedding"].iloc[0][:5])
print("   → Vector length:", len(pipeline.chunk_df["embedding"].iloc[0]))

📊 Sample embedding (first 5 dims): [-0.042777352035045624, 0.025624385103583336, -0.07883816957473755, 0.022501634433865547, -0.009484909474849701]
   → Vector length: 384


In [39]:
# Dataframe Preview
print("chunk_df shape:", pipeline.chunk_df.shape)
print("Columns:", list(pipeline.chunk_df.columns))
print("Sample metadata row:")
print(pipeline.chunk_df[["chunk_id", "source_row", "Complaint ID", "Product", "Issue"]].sample(1))

chunk_df shape: (628990, 9)
Columns: ['source_row', 'chunk_id', 'chunk_text', 'word_count', 'Complaint ID', 'Product', 'Issue', 'Sub-issue', 'embedding']
Sample metadata row:
        chunk_id  source_row  Complaint ID      Product  \
198443         1      117267       8130916  Credit card   

                                                  Issue  
198443  Problem with a purchase shown on your statement  


In [ ]:
#save all the Dataframes and Indexes for future query handling
# Save the FAISS index
pipeline.save_index("../data/processed/complaints_index.faiss")

# Save the chunked DataFrame with embeddings + metadata
pipeline.save_chunk_df("../data/processed/chunked_complaints.csv")

# Save the filtered (post-cleaning) complaint narratives
pipeline.filtered_df.to_csv("../data/processed/final_filtered_complaints.csv", index=False)
print("Dataframes and Indexes saved successfully")

Dataframes and Indexes saved successfully


In [41]:
# Let's look at a demo illustration of Querry handling
# Example querry 
query = "I was charged twice for a payment I never made"
# Feed to pipline
# The Pipline will encode the querry-> create normalized vecto
# Search Fassi index
# Return the top % semantically similar chunks as a dataframe
results = pipeline.search(query, top_k=5)
#inspect top matches
print(results[["chunk_text", "Product", "Issue", "similarity"]].iloc[0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

chunk_text    double charged me twice in the same day for on...
Product                                         Savings account
Issue                                       Managing an account
similarity                                             0.748645
Name: 161892, dtype: object


In [42]:
# Or preview all matches
for i, row in results.iterrows():
    print(f"\n🧩 Match {i+1} — Similarity: {row['similarity']:.4f}")
    print(f"Issue: {row['Issue']}")
    print(f"Product: {row['Product']}")
    print("Chunk:", row["chunk_text"][:300], "...")


🧩 Match 161893 — Similarity: 0.7486
Issue: Managing an account
Product: Savings account
Chunk: double charged me twice in the same day for one bill. asked my bank to stop on payment. i was told to call but they never reimbursed my funds. ...

🧩 Match 139489 — Similarity: 0.7375
Issue: Billing disputes
Product: Credit card
Chunk: . refuse to do anything about the charge. i think this is an abusive practice. for a missed payment of about $ where i never got the statement i have already paid $ in fees and $ in interest, then on an over paid account more interest is charged. ...

🧩 Match 38181 — Similarity: 0.7355
Issue: Problem with a purchase shown on your statement
Product: Credit card
Chunk: . i explained the situation and they informed me im past the date to dispute!!!! i do not believe i should be penalized for a charge i 1 did not make and 2 was informed not to make any payments. please help. ...

🧩 Match 344775 — Similarity: 0.7308
Issue: Problem with a lender or other company cha

# Text Chunking, Embedding, and Vector Store Indexing Notebook Summary

This notebook demonstrates the full pipeline for transforming cleaned consumer complaint narratives into a searchable vector database, enabling efficient semantic search over complaint data. The process leverages modern NLP and vector search libraries to chunk, embed, index, and query complaint texts.

## 1. Environment Setup

All required dependencies are installed at the start, including:
- `pandas` and `numpy` for data handling,
- `sentence-transformers` for generating dense text embeddings,
- `langchain` for advanced text chunking,
- `faiss-cpu` for fast vector similarity search.

## 2. Data Loading and Pipeline Initialization

The notebook loads the preprocessed complaints dataset (`filtered_complaints.csv`), which contains normalized complaint narratives and relevant metadata. The custom `ComplaintVectorPipeline` class is instantiated, encapsulating all logic for chunking, embedding, and indexing.

## 3. Text Chunking

Using the pipeline, each complaint narrative is split into overlapping text chunks (default: 800 characters with 100-character overlap). Only complaints with at least 20 words are included, ensuring meaningful content in each chunk. The resulting `chunk_df` contains one row per chunk, with metadata such as the original complaint ID, product, issue, and chunk order. The shape and sample rows of this chunked DataFrame are displayed for verification.

## 4. Embedding and Indexing

Each chunk is embedded into a 384-dimensional vector using the `all-MiniLM-L6-v2` model from Sentence Transformers. Embeddings are normalized for cosine similarity. The notebook previews the first few dimensions of a sample embedding and confirms the expected vector length. All chunk embeddings are then indexed using FAISS (`IndexFlatIP`), enabling efficient inner product (cosine) similarity search. The notebook verifies the index type, total vectors indexed, and vector dimensionality.

## 5. Saving Artifacts

To support future querying and reproducibility, the following artifacts are saved:
- The FAISS index (`complaints_index.faiss`)
- The chunked DataFrame with embeddings and metadata (`chunked_complaints.csv`)
- The filtered complaints DataFrame (`final_filtered_complaints.csv`)

## 6. Semantic Search Demo

A sample query ("I was charged twice for a payment I never made") is encoded and searched against the FAISS index. The pipeline returns the top 5 most semantically similar complaint chunks, displaying their text, product, issue, and similarity score. The notebook also iterates through all matches, printing their details for inspection.

---

## Key Takeaways

- The notebook provides a robust, end-to-end workflow for transforming complaint narratives into a vector-searchable format.
- All steps, from chunking to embedding and indexing, are automated and reproducible.
- The resulting system enables fast, semantic search over large collections of complaint data, supporting advanced analytics and retrieval use cases in Financial Services